In [8]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-01-1", end="2024-02-26", interval="15m")
dataF.iloc[-1:,:]

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-02-23 22:15:00,1.082251,1.082251,1.082251,1.082251,1.082251,0


In [9]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open > close and previous_open < previous_close and
        close < previous_open and open >= previous_close):
        return 1
    
    # Bullish Pattern
    elif (open < close and previous_open > previous_close
          and close > previous_open and open <= previous_close):
        return 2

    # No Clear Pattern
    else:
        return 0

In [10]:
signal = []
signal.append(0)

for i in range(1, len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))

# sinal_generator(data)
dataF["signal"] = signal

In [11]:
dataF.signal.value_counts()
# dataF.iloc[:, :]

signal
0    3429
2     143
1     142
Name: count, dtype: int64

# 3 - Connect to the market and execute trades

In [12]:
import oandapyV20.endpoints.orders as orders
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
account_id = os.getenv("ACCOUNT_ID")

def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles
    
candles = get_candles(3)

for candle in candles:
    print(float(str(candle.bid.o)) > 1)
    

True
True
True


In [15]:
import pandas as pd

def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=["Open", "Close", "High", "Low"])
    i = 0
    
    for candle in candles:
        dfstream.loc[i, ["Open"]] = float(str(candle.bid.o))
        dfstream.loc[i, ["Close"]] = float(str(candle.bid.c))
        dfstream.loc[i, ["High"]] = float(str(candle.bid.h))
        dfstream.loc[i, ["Low"]] = float(str(candle.bid.l))
        i = i + 1
    
    dfstream.loc[i, ["Open"]] = dfstream["Open"].astype(float)
    dfstream.loc[i, ["Close"]] = dfstream["Close"].astype(float)
    dfstream.loc[i, ["High"]] = dfstream["High"].astype(float)
    dfstream.loc[i, ["Low"]] = dfstream["Low"].astype(float)
    
    signal = signal_generator(dfstream.iloc[:-1,:])
    client = API(access_token)
    SLTPRatio = 2.
    previous_candleR = abs(dfstream["High"].iloc[-2] - dfstream["Low"].iloc[-2])
    
    SLBuy = float(str(candle.bid.o)) - previous_candleR
    SLSell = float(str(candle.bid.o)) + previous_candleR
    
    TPBuy = float(str(candle.bid.o)) + previous_candleR * SLTPRatio
    TPSell = float(str(candle.bid.o)) - previous_candleR * SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)
    
    # Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(account_id, data=mo.data)
        rv = client.request(r)
        print(rv)
        
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(account_id, data=mo.data)
        rv = client.request(r)
        print(rv)

# 4 - Executing orders automatically with a scheduler

In [17]:
# trading_job()

scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2024-02-26 12:00:00', timezone='America/Chicago')
scheduler.start()

KeyboardInterrupt: 